In [75]:
import time
import requests
import json
import pandas as pd
import urllib.parse
from pathlib import Path
from datetime import date
from datetime import timedelta

# GLOBAL VARIABLES #
AFF_SYNC_COLUMNS = ['affiliateid','affiliateein','affiliatepercapitapin','IsChartered','CharterDate','electionmonth','officertermstartmonth','iselectionyearodd','electiontermyears','updatedby','updatedat']
APC_SYNC_COLUMNS = ['AffiliatePerCapitaId','FiscalYearEndMonth','FiscalYearEndDay','PayPerCapitaToAFT','InvoicedByAFT','IncludeAFLCIOPerCapita','AFLCIOAmount','AffiliateBillingFrequencyId','HasOccupationalLiabilityInsurance','FiduciaryBondCoverageID','AccidentInsuranceUnits','ConventionDelegationEligibility','IsAgencyFee','IsStateDues','DeliveryType','GroupNumber','UpdatedBy','UpdatedAt','DeletedAt']
ADIV_SYNC_COLUMNS = ['Action','AffiliateDivisionId','Affiliate','Division','CreatedBy','CreatedAt','UpdatedBy','UpdatedAt','DeletedAt']

YESTERDAY_DATE = (date.today() - timedelta(days = 1)).strftime("%m/%d/%Y")
DATE_TO_RUN = YESTERDAY_DATE

TABLES = ['Affiliate','Division','AffiliateDivision','AffiliatePerCapita','Accounts','StateFederation','AffiliateType','AffiliateDesignation','AffiliateGeoReach']

# JSON PRINT HELPER #
def jprint(output):
    print(json.dumps(output, indent=4))

class KnackAFT:
    def __init__(self):
        # API #
        self.API_KEY = '1b8065b3-d5a5-4586-946a-d9f5d315963f'
        self.APP_ID = '6157aca138a38604ae371cd9'
        
        # HTTP REQUESTS #
        self.GET_HEADERS = {'X-Knack-REST-API-KEY':self.API_KEY,'X-Knack-Application-Id':self.APP_ID}
        self.POST_HEADERS = {'X-Knack-REST-API-KEY':self.API_KEY,'X-Knack-Application-Id':self.APP_ID,'content-type':'application/json'}
        self.API_URL = f'https://api.knack.aft.org/v1/'
        self.LOADER_URL = f'https://loader.knack.aft.org/v1/applications/{self.APP_ID}'

        # INTERNAL #
        self.APP_DICT = {}

        res = requests.get(url=self.LOADER_URL)
        objects = res.json()['application']['objects']

        for obj in objects:
            fields = {}
            name = obj['name']
            key = obj['key']
            
            if name in TABLES:
                for item in obj['fields']:
                    fields.update({item['name']:item['key']})
                self.APP_DICT.update({name:{'obj_id':key,'fields':fields}})
        
    # function to return key for any value
    def get_key(self, dictionary ,val):
        for key, value in dictionary.items():
            if val == value:
                return key

        return ''
        
    
    # GET and format json from requestURL
    def getJSON(self, url):
        r = requests.get(url = self.API_URL + url, headers = self.GET_HEADERS)
        return r.json()
    
    def getObjectJSON(self, object_name):
        return (self.getJSON('objects/' + self.APP_DICT[object_name]['obj_id']))['object']
        
    def find_matches(self, object_name, field_name, match_val):
        field_id = self.APP_DICT[object_name]['fields'][field_name]
        object_id = self.APP_DICT[object_name]['obj_id']
        
        match_filter = {'match':'and', 'rules':[{'field':field_id, 'operator':'is', 'value': match_val}]}
        filter_for_url = urllib.parse.quote(json.dumps(match_filter))
        request_url = "objects/" + object_id + "/records?filters=" + filter_for_url
        res = self.getJSON(request_url)
        if res["total_records"] == 0:
            return ''
        else:
            return res["records"]
        
    def find_records_updated_at_date(self, knack_object, date):
        #Convert to IDs
        knack_object_id = self.APP_DICT[knack_object]['obj_id']
        field_to_match_id = self.APP_DICT[knack_object]['fields']['mdate']
        aftdbupdate_id = self.APP_DICT[knack_object]['fields']['AFTDBUpdate']

        #Get Id {'field':field_to_match_id, 'operator':'is', 'value':date}, 
        match_filter = {'match':'and', 'rules':[{'field':aftdbupdate_id, 'operator':'is', 'value':0}]}
        filter_for_url = urllib.parse.quote(json.dumps(match_filter))
        request_url = "https://api.knack.aft.org/v1/objects/" + knack_object_id + "/records?filters=" + filter_for_url + "&rows_per_page=1000"

        r = requests.get(url = request_url, headers = self.GET_HEADERS)
        #print(json.dumps(r.json(), indent=4))
        res_json_dict = json.loads(json.dumps(r.json()))
        if res_json_dict["total_records"] == 0:
            return ''
        else:
            return res_json_dict["records"]
        
    def get_userid(self, user):
        #Convert to IDs
        knack_object_id = self.APP_DICT['Accounts']['obj_id']
        knack_field_id = self.APP_DICT['Accounts']['fields']['KnackUserID']

        request_url = "https://api.knack.aft.org/v1/objects/" + knack_object_id + "/records/" +  user
        r = requests.get(url = request_url, headers = self.GET_HEADERS)
        #print(json.dumps(r.json(), indent=4))
        res_json_dict = json.loads(json.dumps(r.json()))
        return res_json_dict[knack_field_id]
    
    def get_connection(self,connection_name,field,connection):
        #Convert to IDs
        knack_object_id = self.APP_DICT[connection_name]['obj_id']
        knack_field_id = self.APP_DICT[connection_name]['fields'][field]
        
        request_url = "https://api.knack.aft.org/v1/objects/" + knack_object_id + "/records/" +  connection
        r = requests.get(url = request_url, headers = self.GET_HEADERS)
        res_json_dict = json.loads(json.dumps(r.json()))
        if res_json_dict:
            return res_json_dict[knack_field_id]
        else:
            return ''


client = KnackAFT()
aff_fields = {}
apc_fields = {}
adiv_fields = {}

#Temp Fixes for naming convention, will address permenantely in API Wrapper 2.0
aff_temp = client.APP_DICT['Affiliate']['fields']
aff_temp.update({
    'affiliateein':aff_temp.pop('Affiliate EIN'),
    'affiliatepercapitapin':aff_temp.pop('Affiliate Percapita Pin'),
    'IsChartered':aff_temp.pop('Is Affiliate Chartered'),
    'CharterDate':aff_temp.pop('Charter Date'),
    'ParentAffiliateID':aff_temp.pop('State Federation'),
    'AffiliateTypeID':aff_temp.pop('Affiliate Type'),
    'AffiliateDesignationID':aff_temp.pop('Affiliate Designation'),
    'AffiliateGeoReachID':aff_temp.pop('Affiliate Geographical Reach'),
    'LocationStateAbr':aff_temp.pop('Location State')
})

for name in AFF_SYNC_COLUMNS:
    aff_fields.update({name:client.APP_DICT['Affiliate']['fields'][name]})
        
for name in APC_SYNC_COLUMNS:
    apc_fields.update({name:client.APP_DICT['AffiliatePerCapita']['fields'][name]})
    
for name in ADIV_SYNC_COLUMNS:
    adiv_fields.update({name:client.APP_DICT['AffiliateDivision']['fields'][name]})

def affiliate_runner():
    output_df = []
    records = client.find_records_updated_at_date('Affiliate', DATE_TO_RUN)
    if records:
        for record in records:
            output = {}
            for k,v in aff_fields.items():
                output.update({k:str(record[v])})
                
            user_conn = record[client.APP_DICT['Affiliate']['fields']['muser']+'_raw'][0]['id']
            mdate = record[client.APP_DICT['Affiliate']['fields']['mdate']]
            muser = client.get_userid(user_conn)

            checker = record[client.APP_DICT['Affiliate']['fields']['ParentAffiliateID']+'_raw']
            if checker:
                val = client.get_connection('StateFederation','AffiliateId',checker[0]['id'])
                output.update({'ParentAffiliateID':val})
            else:
                output.update({'ParentAffiliateID':''})

            val = client.get_connection('AffiliateType','AffiliateTypeId',record[client.APP_DICT['Affiliate']['fields']['AffiliateTypeID']+'_raw'][0]['id'])
            output.update({'AffiliateTypeID':val})
            
            val = client.get_connection('AffiliateDesignation','AffiliateDesignationId',record[client.APP_DICT['Affiliate']['fields']['AffiliateDesignationID']+'_raw'][0]['id'])
            output.update({'AffiliateDesignationID':val})
            
            val = client.get_connection('AffiliateGeoReach','AffiliateGeoReachId',record[client.APP_DICT['Affiliate']['fields']['AffiliateGeoReachID']+'_raw'][0]['id'])
            output.update({'AffiliateGeoReachID':val})
            
            output.update({'updatedat':str(mdate)})
            output.update({'updatedby':str(muser)})

            affiliatename = record[client.APP_DICT['Affiliate']['fields']['AffiliateName']]
            output.update({'affiliatename':str(affiliatename)})
            
            lstateab = ''
            loc_rec = record[client.APP_DICT['Affiliate']['fields']['LocationStateAbr']+'_raw']
            if len(loc_rec) == 1:
                lstateab = loc_rec[0]['identifier']
            output.update({'locationstateabr':str(lstateab)})
            
            output_df.append(output)

    #return pd.DataFrame(output_df, dtype=str)
    jprint(output_df)
    
def affiliatepercapita_runner():
    output_df = []
    records = client.find_records_updated_at_date('AffiliatePerCapita', DATE_TO_RUN)
    if records:
        for record in records:
            output = {}
            for k,v in apc_fields.items():
                output.update({k:str(record[v])})
                
            user_conn = record[client.APP_DICT['AffiliatePerCapita']['fields']['muser']+'_raw'][0]['id']
            mdate = record[client.APP_DICT['AffiliatePerCapita']['fields']['mdate']]
            muser = client.get_userid(user_conn)
            
            output.update({'UpdatedAt':str(mdate)})
            output.update({'UpdatedBy':str(muser)})
            output_df.append(output)
            
    #return pd.DataFrame(output_df, dtype=str)
    jprint(output_df)

def affiliatedivision_runner():
    output_df = []
    records = client.find_records_updated_at_date('AffiliateDivision', DATE_TO_RUN)
    if records:
        for record in records:
            output = {}
            for k,v in adiv_fields.items():
                output.update({k:str(record[v])})
            
            user_conn = record[client.APP_DICT['AffiliateDivision']['fields']['muser']+'_raw'][0]['id']
            mdate = record[client.APP_DICT['AffiliateDivision']['fields']['mdate']]
            muser = client.get_userid(user_conn)
            
            val = client.get_connection('Affiliate','affiliateid',record[client.APP_DICT['AffiliateDivision']['fields']['Affiliate']+'_raw'][0]['id'])
            output.update({'Affiliate':val})

            val = client.get_connection('Division','DivisionId',record[client.APP_DICT['AffiliateDivision']['fields']['Division']+'_raw'][0]['id'])
            output.update({'Division':val})
            
            output.update({'UpdatedAt':str(mdate)})
            output.update({'UpdatedBy':str(muser)})
            out_sort = {}
            for k,v in output.items():
                if k == 'Affiliate':
                    out_sort.update({'AffiliateID':v})
                elif k == 'Division':
                    out_sort.update({'DivisionID':v})
                else:
                    out_sort.update({k:v})
            if out_sort['Action'] == 'Delete':
                if out_sort['AffiliateDivisionId']:
                    output_df.append(out_sort)
            else:
                output_df.append(out_sort)
            
    #return pd.DataFrame(output_df, dtype=str)
    jprint(output_df)
    
#jprint(client.APP_DICT)
affiliatedivision_runner()
#affiliatepercapita_runner()
#affiliate_runner()

[
    {
        "Action": "Insert",
        "AffiliateDivisionId": "",
        "AffiliateID": 3111,
        "DivisionID": 3,
        "CreatedBy": "60",
        "CreatedAt": "02/23/2023",
        "UpdatedBy": "19",
        "UpdatedAt": "02/23/2023 5:47pm",
        "DeletedAt": ""
    },
    {
        "Action": "Insert",
        "AffiliateDivisionId": "",
        "AffiliateID": 2254,
        "DivisionID": 6,
        "CreatedBy": "60",
        "CreatedAt": "02/22/2023",
        "UpdatedBy": "19",
        "UpdatedAt": "02/22/2023 2:32pm",
        "DeletedAt": ""
    },
    {
        "Action": "Insert",
        "AffiliateDivisionId": "",
        "AffiliateID": 10530,
        "DivisionID": 3,
        "CreatedBy": "60",
        "CreatedAt": "02/15/2023",
        "UpdatedBy": "19",
        "UpdatedAt": "02/15/2023 8:14pm",
        "DeletedAt": ""
    },
    {
        "Action": "Insert",
        "AffiliateDivisionId": "",
        "AffiliateID": 180,
        "DivisionID": 3,
        "CreatedBy":

In [2]:
import requests
import json
import pandas as pd
import urllib.parse
import time
from pathlib import Path
from datetime import date
from datetime import timedelta

# GLOBAL VARIABLES #
TABLES = ['Affiliate','AffiliatePerCapita','AffiliateDivision','Division','Accounts','StateFederation','EntityType','Geographic State','AffiliateType','FiduciaryBondCoverage','AffiliateDesignation','AffiliateInactiveReason','AffiliateGeoReach', 'Region']

# JSON PRINT HELPER #
def jprint(output):
    print(json.dumps(output, indent=4))

class KnackAFT:
    def __init__(self):
        # API #
        self.API_KEY = '1b8065b3-d5a5-4586-946a-d9f5d315963f'
        self.APP_ID = '6157aca138a38604ae371cd9'
        
        # HTTP REQUESTS #
        self.GET_HEADERS = {'X-Knack-REST-API-KEY':self.API_KEY,'X-Knack-Application-Id':self.APP_ID}
        self.POST_HEADERS = {'X-Knack-REST-API-KEY':self.API_KEY,'X-Knack-Application-Id':self.APP_ID,'content-type':'application/json'}
        self.API_URL = f'https://api.knack.aft.org/v1/'
        self.LOADER_URL = f'https://loader.knack.aft.org/v1/applications/{self.APP_ID}'

        # INTERNAL #
        self.APP_DICT = {}

        res = requests.get(url=self.LOADER_URL)
        objects = res.json()['application']['objects']

        for obj in objects:
            fields = {}
            name = obj['name']
            key = obj['key']

            if name in TABLES:
                for item in obj['fields']:
                    fields.update({item['name'].lower():item['key']})
                self.APP_DICT.update({name.lower():{'obj_id':key,'fields':fields}})
        
    # function to return key for any value
    def get_key(self, dictionary ,val):
        for key, value in dictionary.items():
            if val == value:
                return key

        return ''
        
    
    # GET and format json from requestURL
    def getJSON(self, url):
        r = requests.get(url = self.API_URL + url, headers = self.GET_HEADERS)
        return r.json()
    
    def getObjectJSON(self, object_name):
        return (self.getJSON('objects/' + self.APP_DICT[object_name]['obj_id']))['object']

    def find_matches(self, object_name, field_name, match_val):
        field_id = self.APP_DICT[object_name]['fields'][field_name]
        object_id = self.APP_DICT[object_name]['obj_id']
        
        match_filter = {'match':'and', 'rules':[{'field':field_id, 'operator':'is', 'value': match_val}]}
        filter_for_url = urllib.parse.quote(json.dumps(match_filter))
        request_url = "objects/" + object_id + "/records?filters=" + filter_for_url
        res = self.getJSON(request_url)
        if res["total_records"] == 0:
            return ''
        else:
            return res["records"]
        
    def get_userid(self, user):
        #Convert to IDs
        knack_object_id = self.APP_DICT['Accounts']['obj_id']
        knack_field_id = self.APP_DICT['Accounts']['fields']['KnackUserID']

        request_url = "https://api.knack.aft.org/v1/objects/" + knack_object_id + "/records/" +  user
        r = requests.get(url = request_url, headers = self.GET_HEADERS)
        #print(json.dumps(r.json(), indent=4))
        res_json_dict = json.loads(json.dumps(r.json()))
        return res_json_dict[knack_field_id]
    
    def get_dual_connection(self,obj_name,field_name1,connection1,field_name2,connection2):
        #Convert to IDs
        knack_object_id = self.APP_DICT[obj_name]['obj_id']
        knack_field_id1 = self.APP_DICT[obj_name]['fields'][field_name1]
        knack_field_id2 = self.APP_DICT[obj_name]['fields'][field_name2]

        #Get Id
        match_filter = {'match':'and', 'rules':[{'field':knack_field_id1, 'operator':'is', 'value': connection1},{'field':knack_field_id2, 'operator':'is', 'value': connection2}]}
        filter_for_url = urllib.parse.quote(json.dumps(match_filter))
        request_url = "https://api.knack.aft.org/v1/objects/" + knack_object_id + "/records?filters=" + filter_for_url
        r = requests.get(url = request_url, headers = self.GET_HEADERS)
        #print(json.dumps(r.json(), indent=4))
        res_json_dict = json.loads(json.dumps(r.json()))
        if res_json_dict["total_records"] == 0:
            return ''
        elif res_json_dict["total_records"] == 1:
            return res_json_dict["records"][0]["id"]
        else:
            return ''
    
    def get_connection(self,obj_name,field_name,connection):
        time.sleep(1)
        #Convert to IDs
        knack_object_id = self.APP_DICT[obj_name]['obj_id']
        knack_field_id = self.APP_DICT[obj_name]['fields'][field_name]

        #Get Id
        match_filter = {'match':'and', 'rules':[{'field':knack_field_id, 'operator':'is', 'value': connection}]}
        filter_for_url = urllib.parse.quote(json.dumps(match_filter))
        request_url = "https://api.knack.aft.org/v1/objects/" + knack_object_id + "/records?filters=" + filter_for_url
        r = requests.get(url = request_url, headers = self.GET_HEADERS)
        #print(json.dumps(r.json(), indent=4))
        res_json_dict = json.loads(json.dumps(r.json()))
        if res_json_dict["total_records"] == 0:
            return ''
        elif res_json_dict["total_records"] == 1:
            return res_json_dict["records"][0]["id"]
        else:
            return ''

def add_affiliate(record):
    aff_dict = client.APP_DICT['affiliate']['fields']
    aff_dict.update({
        'affiliateein':aff_dict.pop('affiliate ein'),
        'affiliatepercapitapin':aff_dict.pop('affiliate percapita pin'),
        'ischartered':aff_dict.pop('is affiliate chartered'),
        'charterdate':aff_dict.pop('charter date'),
        'parentaffiliateid':aff_dict.pop('state federation'),
        'affiliatetypeid':aff_dict.pop('affiliate type'),
        'affiliatedesignationid':aff_dict.pop('affiliate designation'),
        'affiliategeoreachid':aff_dict.pop('affiliate geographical reach'),
        'locationstateabr':aff_dict.pop('location state'),
        'affiliateguid':aff_dict.pop('affiliate guid'),
        'affiliatenumber':aff_dict.pop('affiliate number'),
        'affiliateabbreviatedname':aff_dict.pop('affiliate abbreviation'),
        'affiliateacronym':aff_dict.pop('affiliate acronym'),
        'isaffiliateactive':aff_dict.pop('is affiliate active'),
        'affiliateinactivereason':aff_dict.pop('affiliate inactive reason'),
        'affiliateinactivedate':aff_dict.pop('affiliate inactive date'),
        'regionid':aff_dict.pop('region'),
    })
    
    
    connection_fields = ["retireeentitytypeid", "parentaffiliateid","affiliatetypeid","affiliatedesignationid","affiliategeoreachid","affiliateinactivereasonid", "locationstateabr", "regionid"]
    out = {}
    
    for k,v in record.items():
        if k in connection_fields:
            if k == 'affiliateinactivereasonid':
                conn = client.get_connection('affiliateinactivereason','affiliateinactivereasonid',v)
                out.update({aff_dict['affiliateinactivereason']:conn})
            elif k == 'parentaffiliateid':
                conn = client.get_connection('statefederation','affiliateid',v)
                out.update({aff_dict[k]:conn})
            elif k == 'locationstateabr':
                conn = client.get_connection('geographic state', 'geographic state name', v)
                out.update({aff_dict[k]:conn})
            elif k == 'retireeentitytypeid':
                conn = client.get_connection('entitytype', 'entitytypeid', v)
                out.update({aff_dict[k]:conn})
            else:
                conn = client.get_connection(k.replace('id',''),k,v)
                out.update({aff_dict[k]:conn})

        else:
            out.update({aff_dict[k]:v})

    out.update({aff_dict['aftdbupdate']:1})
    check_exists = client.get_connection('affiliate', 'affiliateid', record['affiliateid'])
    if check_exists:
        print(record['affiliateid'] + " exists! updating record...")
        request_url = "https://api.knack.aft.org/v1/objects/" + client.APP_DICT["affiliate"]['obj_id'] + "/records/" +  check_exists
        r = requests.put(url = request_url, headers = client.POST_HEADERS, data = json.dumps(out))
        print(r)
        if r.status_code != 200 and r.status_code != 500:
            print(json.dumps(r.json(), indent=4))
    else:
        print("Creating new affiliate with id: " + record['affiliateid'])
        request_url = "https://api.knack.aft.org/v1/objects/" + client.APP_DICT["affiliate"]['obj_id'] + "/records"
        r = requests.post(url = request_url, headers = client.POST_HEADERS, data = json.dumps(out))
        print(r)
        if r.status_code != 200 and r.status_code != 500:
            print(json.dumps(r.json(), indent=4))
        
        
def add_affiliatepercapita(record):
    record.pop('includestatepercapita')
    apc_dict = client.APP_DICT['affiliatepercapita']['fields']
    connection_fields = ['affiliateid', 'currentfiduciarybondcoverageid']
    out = {}
    
    for k,v in record.items():
        if k in connection_fields:
            if k == 'affiliateid':
                conn = client.get_connection('affiliate','affiliateid',v)
                out.update({apc_dict['affiliate']:conn})
            else:
                conn = client.get_connection('fiduciarybondcoverage','fiduciarybondcoverageid',v)
                out.update({apc_dict['fiduciarybondcoverage']:conn})

        else:
            out.update({apc_dict[k]:v})

    if not out['field_658']:
        out.update({'field_658':'DONTSENDINVOICE'})
        
    out.update({apc_dict['aftdbupdate']:1})
    check_exists = client.get_connection('affiliatepercapita', 'affiliatepercapitaid', record['affiliatepercapitaid'])
    if check_exists:
        print(record['affiliatepercapitaid'] + " exists! updating record...")
        request_url = "https://api.knack.aft.org/v1/objects/" + client.APP_DICT["affiliatepercapita"]['obj_id'] + "/records/" +  check_exists
        r = requests.put(url = request_url, headers = client.POST_HEADERS, data = json.dumps(out))
        print(r)
        if r.status_code != 200 and r.status_code != 500:
            print(json.dumps(r.json(), indent=4))
    else:
        print("Creating new affiliatepercapita with id: " + record['affiliatepercapitaid'])
        request_url = "https://api.knack.aft.org/v1/objects/" + client.APP_DICT["affiliatepercapita"]['obj_id'] + "/records"
        r = requests.post(url = request_url, headers = client.POST_HEADERS, data = json.dumps(out))
        print(r)
        if r.status_code != 200 and r.status_code != 500:
            print(json.dumps(r.json(), indent=4))
        
def add_affiliatedivision(record):
    adiv_dict = client.APP_DICT['affiliatedivision']['fields']
    connection_fields = ["affiliateid","divisionid"]
    out = {}
    aff_conn = ''
    div_conn = ''
    
    for k,v in record.items():
        if k in connection_fields:
            if k == 'affiliateid':
                aff_conn = client.get_connection('affiliate','affiliateid',v)
                out.update({adiv_dict['affiliate']:aff_conn})
            elif k == 'divisionid':
                div_conn = client.get_connection('division','divisionid',v)
                out.update({adiv_dict['division']:div_conn})
        else:
            out.update({adiv_dict[k]:v})

    out.update({adiv_dict['aftdbupdate']:1})

    check_exists = client.get_dual_connection('affiliatedivision', 'affiliate', aff_conn, 'division', div_conn)
    if check_exists:
        print("Affiliate:"+record['affiliateid']+" and Division:"+record['divisionid']+ " found! updating record...")
        request_url = "https://api.knack.aft.org/v1/objects/" + client.APP_DICT["affiliatedivision"]['obj_id'] + "/records/" +  check_exists
        r = requests.put(url = request_url, headers = client.POST_HEADERS, data = json.dumps(out))
        print(r)
        if r.status_code != 200 and r.status_code != 500:
            print(json.dumps(r.json(), indent=4))
    else:
        print("Creating new affiliatedivision with id: " + record['affiliatedivisionid'])
        request_url = "https://api.knack.aft.org/v1/objects/" + client.APP_DICT["affiliatedivision"]['obj_id'] + "/records"
        r = requests.post(url = request_url, headers = client.POST_HEADERS, data = json.dumps(out))
        print(r)
        if r.status_code != 200 and r.status_code != 500:
            print(json.dumps(r.json(), indent=4))

client = KnackAFT()
#jprint(client.APP_DICT)
test_aff = {
    'affiliateid':'999999',
    'affiliateguid':'9999R9999-26FD-47F5-A4C5-999999R99999',
    'affiliatename':'TEST API',
    'affiliatenumber':'9999T',
    'affiliateabbreviatedname':'TEST',
    'affiliateacronym':'TEST',
    'affiliateein':'',
    'affiliatepercapitapin':'99999',
    'billhighwaygroupid':'112907',
    'ischartered':'0',
    'charterdate':'10/9/1997',
    'parentaffiliateid':'49',
    'affiliatetypeid':'4',
    'affiliatedesignationid':'1',
    'affiliategeoreachid':'2',
    'isaffiliateactive':'0',
    'affiliateinactivereasonid':'1',
    'affiliateinactivedate':'10/9/1997',
    'locationstateabr':'VA',
    'regionid':'5',
    'retireeentitytypeid':'1',
    'retireedestinationid':'3150',
    'electionmonth':'6',
    'officertermstartmonth':'2',
    'iselectionyearodd':'1',
    'electiontermyears':'3',
    'noncoaupdate':'0',
    'nonationalupdate':'0',
    'nostateupdate':'0',
    'nolanwanupdate':'0',
    'noexternalupdate':'0',
    'affiliatewebsite':'www.test.org',
    'isactionnetwork':'0',
    'usesaftmemberid':'0',
    'createdby':'0',
    'createdat':'2/28/2023',
    'updatedby':'0',
    'updatedat':'2/28/2023',
    'deletedat':'2/28/2023'
}

test_aff_percap = {
    'affiliatepercapitaid':'99999999',
    'affiliateid':'999999',
    'fiscalyearendmonth':'12',
    'fiscalyearendday':'28',
    'paypercapitatoaft':'0',
    'invoicedbyaft':'0',
    'includestatepercapita':'0',
    'includeaflciopercapita':'1',
    'aflcioamount':'0.00',
    'affiliatebillingfrequencyid':'1',
    'hasoccupationalliabilityinsurance':'0',
    'accidentinsuranceunits':'5',
    'hascope':'0',
    'iscopevoluntary':'0',
    'haspipe':'0',
    'isdirectbargaining':'0',
    'conventiondelegationeligibility':'0',
    'hasstatewebaccess':'1',
    'currentfiduciarybondcoverageid':'8',
    'requestedfiduciarybondcoverageid':'1',
    'customernumber':'LTEST',
    'isagencyfee':'0',
    'isstatedues':'0',
    'isstateaflcio':'0',
    'deliverytype':'SENDINVOICE',
    'insuranceagencyfee':'TBD',
    'groupnumber':'17',
    'createdby':'0',
    'createdat':'2/28/2023',
    'updatedby':'0',
    'updatedat':'2/28/2023',
    'deletedat':'2/28/2023'
}

test_affdivision = {
    'affiliatedivisionid':'999999999',
    'affiliateid':'999999',
    'divisionid':'1',
    'createdby':'0',
    'createdat':'2/28/2023',
    'updatedby':'0',
    'updatedat':'2/28/2023',
    'deletedat':'2/28/2023'
}

add_affiliate(test_aff)
#add_affiliatepercapita(test_aff_percap)
# add_affiliatedivision(test_affdivision)

999999 exists! updating record...
<Response [200]>
